In [ ]:
#Are there patterns over time in non-member and member attendance based on zip code and demographics? (Month to month)
    #Exploratory

#What are membership renewal rates? How many times are guests returning before becoming members?
    #Probably can't answer this question with this data. Data shows membership re-entry, but does not capture return General Admission.

#Model predicted non-member/member attendance considering population increase. What might attendance be like in 10 years?
    #Would need to tie in projected population by county/zip code from another data source. Maybe just focus on this data set for now, tackle population projection if there's time
    ##For modeL:
    ##target variable would be 'member'
    ##assign 1 as member (accesscodegroup '3 Members') and 0 as non-member (accesscodegroup '1 GA')

#What cities should marketing efforts focus on (proportionately to population) to improve guest attendance?

In [ ]:
# Utah population projections by county: https://tableau.dashboard.utah.edu/t/Business/views/20220111_Detailed_Proj_Vis/ProjectionsWorkbook?%3Adisplay_count=n&%3Aembed=y&%3AisGuestRedirectFromVizportal=y&%3Aorigin=viz_share_link&%3AshowAppBanner=false&%3AshowVizHome=n

In [20]:
# Import Dependencies
import pandas as pd

In [21]:
attendance_data_csv = "./Resources/attendance_data_sql.csv"

attendance_df = pd.read_csv(attendance_data_csv)

In [22]:
attendance_df.head()

,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup
0,1,2022-02-22 00:00:00,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
1,1,2022-02-22 00:00:00,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
2,1,2022-02-22 00:00:00,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
3,0,2022-02-22 00:00:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000004,2022-02-22 14:04:00,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis
4,0,2022-02-22 00:00:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000005,2022-02-22 14:04:00,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis


In [23]:
row_count = len(attendance_df)
row_count

943902

In [24]:
attendance_df.dtypes

admits              int64
usedate            object
plu                object
pluname            object
accesscode          int64
accesscodegroup    object
accesscodename     object
usageid             int64
saledatetime       object
salenode            int64
zipcode             int64
city               object
county             object
stateloc           object
subgroup           object
maingroup          object
dtype: object

In [25]:
pd.to_datetime(attendance_df.usedate)

0        2022-02-22
1        2022-02-22
2        2022-02-22
3        2022-02-22
4        2022-02-22
            ...    
943897   2023-03-25
943898   2023-03-25
943899   2023-03-25
943900   2023-03-25
943901   2023-03-25
Name: usedate, Length: 943902, dtype: datetime64[ns]

In [26]:
pd.to_datetime(attendance_df.saledatetime)

0        2022-02-22 13:46:00
1        2022-02-22 13:46:00
2        2022-02-22 13:46:00
3        2022-02-22 14:04:00
4        2022-02-22 14:04:00
                 ...        
943897   2023-03-25 15:46:00
943898   2023-03-25 15:46:00
943899   2023-03-25 15:47:00
943900   2023-03-25 15:47:00
943901   2023-03-25 15:47:00
Name: saledatetime, Length: 943902, dtype: datetime64[ns]

In [27]:
attendance_df.saledatetime.min()

'2000-01-01 00:00:00'

In [28]:
attendance_df.saledatetime.max()

'2023-03-25 15:47:00'

In [34]:
date_filtered_attendance_df = attendance_df[(attendance_df['saledatetime'] > '2021-01-01')]

In [35]:
date_filtered_row_count = len(date_filtered_attendance_df)
date_filtered_row_count

880279

In [36]:
date_filtered_attendance_df.plu.unique()

array(['GA01010TDP          ', 'GA01055TDP          ',
       'GA01060T            ', 'TI0003              ',
       'GA01050TDP          ', 'MB00001             ',
       'MB00011             ', 'TI0002              ',
       'MB00057             ', 'MB00013             ',
       'MB00057R            ', 'MB00015             ',
       'MPR01               ', 'GA01030TDP          ',
       'MPR21               ', 'MB00014-15          ',
       'GA01020TDP          ', 'MB00009             ',
       'GA01040TDP          ', 'MB00012-15          ',
       'TI0001              ', 'MB00057BF           ',
       'COMP04001           ', 'MB00002-15          ',
       'MPR05               ', 'GA01110             ',
       'MPR22               ', 'MPR10               ',
       'MB00055R            ', 'SCTF00001           ',
       'SCTF00001M          ', 'MB00005             ',
       'MB00016-15          ', 'MPR04               ',
       'MB00013BF           ', 'MB00003BF           ',
       'MB

In [37]:
date_filtered_attendance_df.pluname.unique()

array(['Adult Admission', 'Teen Admission', '2 & Under Admission',
       "Tuki's Island Member", 'Child Admission', 'Child Membership',
       'Adult Membership', "Tuki's Island 2 & Under",
       'Quad Family Membership', 'Family Membership',
       'Quad Family Membership Renewal', 'Family Plus Membership',
       'Family Membership Promo', 'Military Admission',
       'Quad Membership Promo', 'Family Membership 4 Child Renewal',
       'Senior Admission', 'Senior Membership', 'Student Admission',
       'Adult Membership Renewal', "Tuki's Island 1 Hr",
       'Quad Family Membership Black Friday', 'All Ages VIP',
       'Child Membership Renewal', 'Adult Membership Promo',
       'Member Promo GA Admission', 'Child Membership Promo',
       'Grandparent Membership Renewal', 'Teen STEM Futures',
       'Student Membership', 'Family Plus Membership Renewal',
       'Grandparent Plus Promo', 'Family Membership Black Friday',
       'Teen Membership Black Friday', 'Adult Membership Bla

In [38]:
date_filtered_attendance_df.accesscode.unique()

array([  108,   450,    11,    21,    20,    10,   150,   100,    24,
         415,   310,   311,    23,   210,   200,   502,   501,   106,
          30, 91000,   411,   420,   107,   412,   300, 12550,   410,
         111,    26,   406,   401,    31,   112,   414, 95110, 20200,
         416,   220, 91001,   413,   407], dtype=int64)

In [39]:
date_filtered_attendance_df.accesscodegroup.unique()

array(['1 GA', '9 Fun Center', '3 Members', '4 Comp', '9 Camp', '2 Group',
       '5 School', '8 EN', 'CHARITABLE', '6 Event', '7 Fac Rental',
       'FAM MEM', 'PARTY', '10 Camp', 'MARKETING', 'VR', 'PROMOTION'],
      dtype=object)

In [40]:
date_filtered_attendance_df.accesscodename.unique()

array(['Timed Ticketing GA            ', "Tuki's Island                 ",
       'Membership Readmit            ', 'Family Membership Readmit     ',
       'Family Membership             ', 'Membership                    ',
       'Comp                          ', 'Aquarium Admission            ',
       'Grandparent Membership Readmit', 'Youth/Family Program          ',
       'Adult                         ', 'Student                       ',
       'Grandparent Membership        ', 'Stingray Feed                 ',
       'Penguin Encounter             ', 'Party Admission               ',
       'Employee Membership           ', 'CHARITABLE VOUCHER            ',
       'Printed Hosted Event          ', 'Facility Rental               ',
       'Member Ticketed Admission     ', 'Aquarium Hosted Event         ',
       'Group                         ', 'MEMBERSHIP - FAMILY WEB       ',
       'Hosted Event                  ', 'Party Room                    ',
       'Circle of Friends

In [42]:
date_filtered_attendance_df.zipcode.nunique()

9849

In [46]:
date_filtered_attendance_df.city.nunique()

5054

In [47]:
date_filtered_attendance_df.county.unique()

array(['Salt Lake', 'Davis', 'Utah', 'Uintah', 'Weber', 'unknown',
       'Cache', 'Summit', 'Juab', 'Morgan', 'Tooele', 'Beaver', 'Grand',
       'Wasatch', 'Box Elder', 'Iron', 'Washington', 'Daggett',
       'Duchesne', 'Sanpete', 'Sevier', 'Kane', 'Millard', 'Garfield',
       'Rich', 'Emery', 'Carbon', 'Wayne', 'Piute', 'San Juan'],
      dtype=object)

In [48]:
date_filtered_attendance_df.stateloc.unique()

array(['UT', 'ID', 'unknown', 'CA', 'WY', 'AZ', 'MO', 'IN', 'WA', 'TX',
       'CO', 'NY', 'NM', 'FL', 'AR', 'SC', 'ME', 'RI', 'PA', 'MI', 'MT',
       'VA', 'MA', 'IL', 'MD', 'IA', 'NV', 'VT', 'NH', 'MN', 'OK', 'HI',
       'NJ', 'OH', 'WI', 'KY', 'AK', 'CT', 'GA', 'ND', 'OR', 'NE', 'TN',
       'NC', 'LA', 'DE', 'AE', 'PR', 'SD', 'KS', 'MS', 'AL', 'AP', 'DC',
       'WV', 'AA', 'VI', 'MP', 'GU'], dtype=object)

In [49]:
date_filtered_attendance_df.subgroup.unique()

array(['Salt Lake City, Taylorsville', 'Bountiful', 'Salt Lake City',
       'Provo', 'West Jordan', 'Vernal', 'Riverton', 'Kaysville',
       'American Fork', 'Sandy', 'Midvale', 'Lehi', 'Ogden', 'Clearfield',
       'Herriman', 'Saratoga Springs', 'Outside Utah', 'Wellsville',
       'South Jordan', 'unknown', 'Draper', 'Orem', 'Park City', 'Nephi',
       'Morgan', 'Pleasant Grove', 'West Jordan, West Valley City',
       'Tooele', 'Eagle Mountain', 'Milford', 'Logan', 'Moab', 'Alpine',
       'Heber City', 'Mantua', 'Syracuse', 'Magna', 'Layton',
       'Cedar City', 'Roy', 'Springville', 'Hurricane', 'Santaquin',
       'Manila', 'Payson', 'Lindon', 'North Salt Lake', 'Hill Afb',
       'Duchesne', 'Altonah', 'Spanish Fork', 'Event, School, etc.',
       'Stockton', 'Salem', 'Altamont', 'Woods Cross', 'Tremonton',
       'Saint George', 'Talmage', 'Ephraim', 'Midway', 'Centerville',
       'Richfield', 'Rush Valley', 'Glendale', 'Delta', 'Santa Clara',
       'Ivins', 'Bryce', 'Ro

In [52]:
date_filtered_attendance_df.maingroup.unique()

array(['Salt Lake County', 'Davis', 'Utah County', 'Other Utah Counties',
       'Out of State', 'unknown'], dtype=object)